## Vectorización de información sobre avisos

Genera un archivo `avisos.csv` que contiene los avisos en su representación vectorial.

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from rutas import *

%matplotlib inline

In [30]:
avisos_detalle = pd.read_csv(RUTA_AVISOS_DETALLE)
# Estas columnas no aportan suficiente información
# idpais: Siempre vale 1
# ciudad: Hay alrededor de 50 registros con esta información, el resto es NaN
# mapacalle: Hay sólo 683 registros con esta información.
avisos_detalle = avisos_detalle.drop(axis=1, labels=['idpais', 'ciudad', 'mapacalle', 'titulo', 'descripcion', 'nivel_laboral', 'denominacion_empresa'])
avisos_detalle['nombre_zona'] = avisos_detalle['nombre_zona'].astype('category')
avisos_detalle.head()

,idaviso,nombre_zona,tipo_de_trabajo,nombre_area
0,1001284385,Corrientes,Full-time,Otros
1,1110092955,GBA Oeste,Full-time,Producción
2,1110092958,GBA Oeste,Full-time,Mantenimiento
3,1110391074,Cordoba,Full-time,Ingeniería Industrial
4,1110669632,GBA Oeste,Full-time,Mantenimiento


#### Determinar la zona por la distancia a Salta y a Bariloche
Dado la distancia a dos puntos y conociendo la distancia entre los puntos se puede generar un triangulo y determinar la posición espacial del tercero

Debido a que pueden generarse dos triángulos iguales (uno para cada lado),
existe la posibilidad de encontrar un falso positivo.
Esta posibilidad se minimiza haciendo que los dos puntos queden de un lado del país
de este modo el tercer punto queda en Argentina, Chile o en el oceano pacífico. Dado el contexto, el único punto que tiene sentido es que quede dentro de Argentina.
zona: (distancia a salta, distancia a bariloche)

In [31]:
distancias_zonas = {
    'Capital Federal': (1460, 1580),
    'Gran Buenos Aires': (1467, 1550),
    'Buenos Aires (fuera de GBA)': (1537, 1209),
    'GBA Oeste': (1471, 1562),
    'La Plata': (1335, 1600),
    'Mendoza': (1260, 1570),
    'Ciudad de Mendoza': (1260, 1570),
    'Rosario': (1462, 1616),
    'Cordoba': (1464, 1535),
    'Tucuman': (305, 1566),
    'Santa Fe': (1462, 1616),
    'Santa Cruz': (3460, 1600),
    'San Juan': (1132, 1380),
    'Neuquen': (1970, 440),
    'Corrientes': (830, 2400),
    'Catamarca': (533, 1866)
}

avisos_detalle['dist_a_salta'] = avisos_detalle['nombre_zona'].map(lambda x: distancias_zonas[x][0])
avisos_detalle['dist_a_brc'] = avisos_detalle['nombre_zona'].map(lambda x: distancias_zonas[x][1])
avisos_detalle = avisos_detalle.drop(axis=1, labels=['nombre_zona'])
avisos_detalle.head()

,idaviso,tipo_de_trabajo,nombre_area,dist_a_salta,dist_a_brc
0,1001284385,Full-time,Otros,830,2400
1,1110092955,Full-time,Producción,1471,1562
2,1110092958,Full-time,Mantenimiento,1471,1562
3,1110391074,Full-time,Ingeniería Industrial,1464,1535
4,1110669632,Full-time,Mantenimiento,1471,1562


#### Separar tipo_de_trabajo en carga_horaria y longevidad_laboral: -1 = temporario; 1 = permanente; 0 = n/a

El tipo de trabajo en general significa dos cosas: la carga horaria del mismo y cuánto tiempo dura el puesto. Una pasantía, por ejemplo, es un trabajo que se sabe que es de duración fija; mientras que un puesto full-time podría ser permanente.

Por otra parte, un puesto full-time implica una carga horaria mínima de 40 horas semanales, mientras que un puesto part-time es de entre 20 y 30 horas semanales.

In [32]:
cargas_horarias = {
    'Full-time': 40,
    'Part-time': 25,
    'Pasantia': 25,
    'Teletrabajo': 30,
    'Por Horas': 20,
    'Temporario': 40,
    'Por Contrato': 40,
    'Fines de Semana': 10,
    'Primer empleo': 40,
    'Voluntario': 40
}

longevidad = {
    'Full-time': 1,
    'Part-time': 1,
    'Pasantia': -1,
    'Teletrabajo': 1,
    'Por Horas': 0,
    'Temporario': -1,
    'Por Contrato': -1,
    'Fines de Semana': 1,
    'Primer empleo': 0,
    'Voluntario': 0
}

avisos_detalle['carga_horaria'] = avisos_detalle['tipo_de_trabajo'].map(lambda x: cargas_horarias[x])
avisos_detalle['longevidad'] = avisos_detalle['tipo_de_trabajo'].map(lambda x: longevidad[x])
avisos_detalle = avisos_detalle.drop(axis=1, labels=['tipo_de_trabajo'])
avisos_detalle.head()

,idaviso,nombre_area,dist_a_salta,dist_a_brc,carga_horaria,longevidad
0,1001284385,Otros,830,2400,40,1
1,1110092955,Producción,1471,1562,40,1
2,1110092958,Mantenimiento,1471,1562,40,1
3,1110391074,Ingeniería Industrial,1464,1535,40,1
4,1110669632,Mantenimiento,1471,1562,40,1


In [33]:
avisos_detalle[avisos_detalle['idaviso'] == 1112491250]

,idaviso,nombre_area,dist_a_salta,dist_a_brc,carga_horaria,longevidad
20934,1112491250,Operaciones,1467,1550,40,1


#### Codificar el área laboral via one-hot
Debido a que no existe una métrica para la distancia entre áreas laborales, la codificación usada es one-hot.

In [34]:
avisos_detalle = pd.get_dummies(avisos_detalle, columns=['nombre_area'])
avisos_detalle.head()

,idaviso,dist_a_salta,dist_a_brc,carga_horaria,longevidad,nombre_area_Abastecimiento,nombre_area_Administracion de Seguros,nombre_area_Administración,nombre_area_Administración de Base de Datos,nombre_area_Administración de Personal,...,nombre_area_Trabajo Social,nombre_area_Trabajo social,nombre_area_Traduccion,nombre_area_Transporte,nombre_area_Turismo,nombre_area_Urbanismo,nombre_area_Venta de Seguros,nombre_area_Ventas,nombre_area_Ventas Internacionales/Exportación,nombre_area_Veterinaria
0,1001284385,830,2400,40,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1110092955,1471,1562,40,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1110092958,1471,1562,40,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1110391074,1464,1535,40,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1110669632,1471,1562,40,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Agregamos información sobre el promedio de edad, sexo y nivel educativo de los postulantes

El sexo puede valer cualquier número en el intervalo [-1, 1], siendo -1 todo el público es femenino, 1 todo el público es masculino y 0 hay equitativamente hombres y mujeres. Los valores intermedios representan proporciones mayoritarias de un sexo u el otro.

El nivel educativo se representa mediante un número entre el 0 y el 10. El 0 significa que el nivel educativo es bajo y el 10 que es alto.

La explicación detallada se encuentra en postulantes.py

In [21]:
postulantes   = pd.read_csv(RUTA_VECTOR_POSTULANTES) # Este archivo se genera en postulantes.py
postulaciones = pd.read_csv(RUTA_SET_ENTRENAMIENTO) # El set tiene filtrado que avisos se vieron y que avisos se postularon.
postulaciones = pd.merge(postulaciones, postulantes, on='idpostulante', how='inner')
postulaciones = postulaciones.drop(axis=1, labels=['idpostulante'])
postulaciones = postulaciones[postulaciones['sepostulo'] == 1]
postulaciones = postulaciones.drop(axis=1, labels=['sepostulo'])
postulaciones.head()

,idaviso,sexo,edad,nivel_educativo
0,1112310051,-1,41,0.0
2,1112204681,-1,41,0.0
3,1112390261,-1,41,0.0
4,1112399940,-1,41,0.0
5,1112260584,-1,41,0.0


In [35]:
# Este factor evita que los anuncios que tienen una sola visita tengan su sexo/genero/edad gobernado por el usuario que lo vio
FACTOR_AMORTIGUACION_PROMEDIO = 5 
avisos_detalle = avisos_detalle.set_index('idaviso')
avisos_detalle = pd.merge(avisos_detalle, postulaciones.groupby('idaviso').agg(lambda x: sum(x) / (len(x) + FACTOR_AMORTIGUACION_PROMEDIO)), 
                          left_index=True, right_index=True, how='left')
avisos_detalle = avisos_detalle.reset_index('idaviso')

In [36]:
avisos_detalle = avisos_detalle.rename(columns={'sexo':'prom_sexo', 'edad':'prom_edad', 'nivel_educativo':'prom_nivel_educativo'})
avisos_detalle['prom_sexo'] = avisos_detalle['prom_sexo'].fillna(0)
avisos_detalle['prom_edad'] = avisos_detalle['prom_edad'].fillna(30)
avisos_detalle['prom_nivel_educativo'] = avisos_detalle['prom_nivel_educativo'].fillna(2)
avisos_detalle.head()

,idaviso,dist_a_salta,dist_a_brc,carga_horaria,longevidad,nombre_area_Abastecimiento,nombre_area_Administracion de Seguros,nombre_area_Administración,nombre_area_Administración de Base de Datos,nombre_area_Administración de Personal,...,nombre_area_Transporte,nombre_area_Turismo,nombre_area_Urbanismo,nombre_area_Venta de Seguros,nombre_area_Ventas,nombre_area_Ventas Internacionales/Exportación,nombre_area_Veterinaria,prom_sexo,prom_edad,prom_nivel_educativo
0,1001284385,830,2400,40,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0.232227,31.454976,2.515403
1,1110092955,1471,1562,40,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0.506747,29.782609,2.069340
2,1110092958,1471,1562,40,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0.647059,29.659229,2.060852
3,1110391074,1464,1535,40,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0.145846,29.793012,2.580947
4,1110669632,1471,1562,40,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0.953748,32.626794,2.213716


In [37]:
# Cachear resultados a disco
avisos_detalle.to_csv(RUTA_DATOS_PROCESADOS + 'avisos_detalle.csv', index=False)